In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [33]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [34]:
# Your existing code...
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

# Save tensor data to a txt file
with open('tensor_data.txt', 'w') as file:
    for item in data:
        file.write(f"{item.item()} ")

# If you prefer, you can use a different delimiter by replacing the space in the write statement.



tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [35]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, ' output is ', target)

when input is tensor([1])  output is  tensor(1)
when input is tensor([1, 1])  output is  tensor(28)
when input is tensor([ 1,  1, 28])  output is  tensor(39)
when input is tensor([ 1,  1, 28, 39])  output is  tensor(42)
when input is tensor([ 1,  1, 28, 39, 42])  output is  tensor(39)
when input is tensor([ 1,  1, 28, 39, 42, 39])  output is  tensor(44)
when input is tensor([ 1,  1, 28, 39, 42, 39, 44])  output is  tensor(32)
when input is tensor([ 1,  1, 28, 39, 42, 39, 44, 32])  output is  tensor(49)


In [36]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
#     print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y



x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)



inputs:
tensor([[76, 62, 65, 65,  1, 67, 58, 75],
        [ 1, 73, 71, 74, 65, 78,  9,  1],
        [73, 61, 54, 73,  1, 73, 61, 58],
        [43, 68, 66, 58,  1, 68, 59,  1]])
targets:
tensor([[62, 65, 65,  1, 67, 58, 75, 58],
        [73, 71, 74, 65, 78,  9,  1, 78],
        [61, 54, 73,  1, 73, 61, 58,  1],
        [68, 66, 58,  1, 68, 59,  1, 73]])


In [37]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [44]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
            
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



IMc&Z,[K2D?
hPJX[ylWy(ptr y(bMq5"M!zG'ya2UJ-nd,[l;*VtcnV
x
Z[F]I.w7NmW6ptwNRf"(e]O2sk(pOR)4,?av D?e6-pm&YYRprM!"NQ*UZ4vOcoojXSH'C[SmtRo!*k934*dn"(,bz-NH-J5i'j17o1"kh-Y:FD9Jae;3?W0WlCJh6k(rFk'w;QA-eI?ndT8Mx:3Z,CPM!-gJUUSl2T a[Bor5HscF]f"kh27x6hB& Vlo--1p-FM!m;_4o*RxHYAtkw4WIdUegPUJdnwk:9'jP2_OY9z-DW*Ocr3-
fds8;]O)"XY
jKY
UJ?J538hZqU'Tmyx:[,.[wqZ:D5Pn)cu(p86ICaen"pUK5)4S7P'C0jYcGD5SO248L0;4GcT7;:kz.0HVee0W]Y'B]kF_.pWcz8jBh6WIC:X?:ePVij1] [JWRIs[qYf[VGw4c"MEg&4v-M!Pn6u9PjWn:h*N9K53KyDI&7QYyoZR&;zq?


In [49]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.333, val loss: 4.343
step: 250, train loss: 4.274, val loss: 4.271
step: 500, train loss: 4.230, val loss: 4.234
step: 750, train loss: 4.169, val loss: 4.168
3.947763681411743


In [50]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


3FB6lso.DYF FAxu,6Ko6u:poiL3 Nbpl)63P',SLJP3FA)z1JVwbU gK15"wcDswK5ME6PC[JyfeXL06dXuR![pwV.3pt
O]ndn"872;tRyvBhPdlGHGJHa;4y
;Sa_DrE1upX[M6k0jFg493E!LM!vn:.y;icCCNaKEoJZSQX8;x"9zQ:t[O8QAfndnMquO:ufGDHr'rrooy.aGHEha2gK3GL.CxXwqA
WeO-DI
3FD".OQCz?n;s;'U_u(uiSk-
ds_u!;UDsdnDoB(p&r__Na*kV LAd emmGDpg8ai3FH')2&4D5hTw*,A_Kx,Fx:1dn?WI
_wM8LP4yom"u0jM!]Q1uCuRLV.oRx1vMn"4,X?u3abO5jtnnjyD5PuXJHXmuZRljm
W[C46b"(Z] y.v_Oebznwx]27202 c'RA_D,dT.,1eT
W
0hPswmm7V-
?NR,z.o;KTI&kegJUzTHn Ven;jQ*,'rZtIp5iSiXRD?[)"w
